# Python para finanças - alocação e otimização de portfólios

### Importação das bibliotecas e base de dados

In [33]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px 

In [39]:
dataset = pd.read_csv('acoes2.csv')
dataset

,Date,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,2015-01-02,14.990000,15.20,5.923076,0.232812,11.910702,47.259998
1,2015-01-05,14.850000,15.00,5.963461,0.237187,11.544731,46.320000
2,2015-01-06,15.210000,14.80,5.875000,0.234062,10.822770,46.580002
3,2015-01-07,14.550000,14.67,5.807692,0.241875,10.746248,48.150002
4,2015-01-08,14.270000,14.15,5.905769,0.240000,10.995774,48.509998
...,...,...,...,...,...,...,...
1448,2020-10-27,18.600000,14.17,41.619999,25.450001,28.500000,95.680000
1449,2020-10-28,16.920000,12.77,39.570000,24.629999,27.500000,91.559998
1450,2020-10-29,16.610001,12.68,40.014999,25.360001,28.500000,92.800003
1451,2020-10-30,15.690000,12.28,37.915001,24.629999,27.000000,90.660004


### Alocação aleatória de ativos

In [40]:
dataset.columns  # Mostra as colunas do DataFrame...

Index(['Date', 'GOL', 'CVC', 'WEGE', 'MGLU', 'TOTS', 'BOVA'], dtype='object')

In [94]:
dataset.loc[len(dataset) - 1 ]['BOVA']

757.1935912425403

In [85]:
def alocacao_ativos(dataset, dinheiro_total, seed=0):
    dataset = dataset.copy()

    if seed != 0:
        np.random.seed(seed)
    
    pesos = np.random.random(len(dataset.columns) - 1)  # len() -> passa a quantidade de colunas, menos uma.
    # print(pesos, pesos.sum()) -> valor maior que 1.0
    pesos = pesos / pesos.sum() 
    # print(pesos, pesos.sum())

    # Normalizando o DataFrame: 'dataset'
    colunas = dataset.columns[1:]
    for i in colunas:
        dataset[i] = (dataset[i] / dataset[i][0])

    for i, acao in enumerate(dataset.columns[1:]):
        """
        O enumerate é responsável em gerar os indices, que neste caso
        são de 0 até 5 para que se possa acessar cada uma das posições 
        do vetor: 'pesos'.

        Args:
            i, acao in enumerate (_type_): _description_
            (i) é o índice e (acao) são os valores de cada ação do DataFrame.
            argumento: 'dataset.columns[1:]' são as colunas do DataFrame: 'dataset'.
        """
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    # Adiciona uma coluna efetuando o somatório total das ações em cada data
    dataset['Soma valor'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']
    # print(datas)
    
    dataset.drop(labels= ['Date'], axis= 1, inplace= True)
    dataset['Taxa retorno'] = 0.0

    # Efetua o calculo da taxa de retorno simples e adiciona o resultado em uma nova coluna
    for i in range(1, len(dataset)):
        dataset['Taxa retorno'][i] = ((dataset['Soma valor'][i] / dataset['Soma valor'][i - 1]) - 1) * 100
    
    acoes_pesos = pd.DataFrame(data = {'Ações': colunas, 'Pesos': pesos * 100})

    return dataset, datas, acoes_pesos, dataset.loc[len(dataset) - 1]['Soma valor']

In [86]:
# Chamando a função
dataset, datas, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes2.csv'), 5000, 10)

In [87]:
dataset

,GOL,CVC,WEGE,MGLU,TOTS,BOVA,Soma valor,Taxa retorno
0,1330.859777,35.806036,1093.315674,1292.008683,860.138954,387.870876,5000.000000,0.000000
1,1318.430187,35.334904,1100.770128,1316.288069,833.710152,380.156147,4984.689588,-0.306208
2,1350.392100,34.863773,1084.441499,1298.945650,781.573275,382.290029,4932.506325,-1.046871
3,1291.795217,34.557537,1072.017409,1342.304468,776.047198,395.175284,4911.897113,-0.417824
4,1266.935952,33.332592,1090.120996,1331.899017,794.066880,398.129839,4914.485276,0.052692
...,...,...,...,...,...,...,...,...
1448,1651.367094,33.379707,7682.460263,141236.798956,2058.145750,785.262100,153447.413870,-0.061512
1449,1502.211333,30.081783,7304.059540,136686.134989,1985.930110,751.448534,148259.866289,-3.380668
1450,1474.688596,29.869773,7386.200159,140737.335967,2058.145750,761.625471,152447.865716,2.824769
1451,1393.007983,28.927508,6998.570287,136686.134989,1949.822290,744.062131,147800.525188,-3.048479


In [88]:
datas

0       2015-01-02
1       2015-01-05
2       2015-01-06
3       2015-01-07
4       2015-01-08
           ...    
1448    2020-10-27
1449    2020-10-28
1450    2020-10-29
1451    2020-10-30
1452    2020-11-03
Name: Date, Length: 1453, dtype: object

In [89]:
acoes_pesos

,Ações,Pesos
0,GOL,26.617196
1,CVC,0.716121
2,WEGE,21.866313
3,MGLU,25.840174
4,TOTS,17.202779
5,BOVA,7.757418


In [92]:
soma_valor

149991.6965915256